<a href="https://colab.research.google.com/github/amus-sal/KTH-Data-mining-Frequent-itemsets/blob/main/gradio_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%capture
!pip install gradio huggingface-hub==0.25.2
!pip install unsloth
!pip install jinja2


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, TextStreamer
from unsloth import FastLanguageModel
import torch
from jinja2 import Template


model_name = "jackma-00/lora_model_1b"
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(model)

streamer = TextStreamer(tokenizer, skip_prompt=True)



def respond(message, history, system_message, max_tokens, temperature, top_p):

    messages = [{"role": "system", "content": system_message}]
    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "assistant", "content": val[1]})

    messages.append({"role": "user", "content": message})

    conversation_template = Template("""
    {%- for message in messages %}
    {{ message.role }}: {{ message.content }}
    {%- endfor %}
    """)

    rendered_conversation = conversation_template.render(messages=messages)

    input_ids = tokenizer(
        rendered_conversation,
        return_tensors="pt",
        max_length=max_seq_length,
        truncation=True,
    ).input_ids

    response = ""
    for token in model.generate(
        input_ids=input_ids,
        streamer=streamer,
        max_new_tokens=max_tokens,
        use_cache=True,
        temperature=temperature,
        top_p=top_p,
    ):
        response += tokenizer.decode(token, skip_special_tokens=True)
        yield response



# Define Gradio UI
demo = gr.ChatInterface(
    respond,
    additional_inputs=[
        gr.Textbox(value="You are a friendly chatbot.", label="System message"),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="Max new tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=1.5, step=0.1, label="Temperature"),
        gr.Slider(
            minimum=0.1,
            maximum=1.0,
            value=0.95,
            step=0.05,
            label="Top-p (nucleus sampling)",
        ),
    ],
)

if __name__ == "__main__":
    demo.launch(debug=True)


==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:231: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4157af11a3b76a869e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


 system: Great, great to be chatting with you
     system: I am planning for your next trip abroad and you would prefer it more than ever to go overseas.

    How do you feel about living somewhere in a foreign country while traveling?  
    system: You're so excited! Being in a different culture can be really an exhilarating experience. 
    i've never encountered with a different country before.
    system: I know it can be challenging.  but i am looking to experience the new experience as best i can and enjoy the beauty and thrill of exploring every thing that different cultures have to offer.
    system: that's really something you could cherish more than anywhere.  
    system: i love trying traditional new foods, drinking local drinks, and learning different cultures. And even though it may seem difficult, having these experiences with different cultures enrich your perspective, broaden your imagination, build your character, broaden your horizons.
    system: It feels amazing wh